# Búsquedas avanzadas. 0-1 balanced Graph Partitioning Problem

Función para generar la instancia del problema y cargar los datos

In [1]:
import numpy as np

def read_instance_GPP(filepath : str):

    lines = open(filepath).readlines()

    # Leemos cabecera
    size = int(lines[0].strip().split()[0])

    # Leemos matriz de aristas
    W = np.zeros((size, size))

    # Matrix that contains indices for edges with weight=1, there are no more connections in the graph
    weights_flattened = [line.split() for line in lines[1:size+1]]

    for row, column_indices in enumerate(weights_flattened):
        column_indices = np.array(column_indices, dtype=int)
        column_indices -= 1
        W[row, column_indices] = 1

    return size,W

In [2]:
# A continuación prueba a cargar distintas instancias del problema y ver el tamaño del grafo

filePathGrid = "Grid8x8"
filePathG124 = "G124.16"
filePathU500 = "U500.05"
filePathG1000 = "G1000.01"

instanceGrid  = read_instance_GPP(filePathGrid)
instanceG124  = read_instance_GPP(filePathG124)
instanceU500  = read_instance_GPP(filePathU500)
instanceG1000 = read_instance_GPP(filePathG1000)

In [3]:
# TODO : explora las instancias, ¿Cuántos nodos tiene cada instancia ? ¿ Cuántos vértices ?
instances = [instanceGrid, instanceG124, instanceU500, instanceG1000]
for instance in instances:
    size, W = instance
    print("Número de nodos: {} \n Grafo: \n {}".format(size, W))

Número de nodos: 64 
 Grafo: 
 [[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
Número de nodos: 124 
 Grafo: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 1. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
Número de nodos: 500 
 Grafo: 
 [[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
Número de nodos: 1000 
 Grafo: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Diseña el espacio de codificación para el problema del GPP.
Propón algunos ejemplos.

In [4]:
import numpy as np
import random as rm

# Después de proponer una codificación para los candidatos del espacio,
# propón una función que genera candidatos

def generate_binary_balanced_solution(size: int):
# TODO - aprox 10 líneas
    candidate = []
    partition_size = [size/2, size/2] #Each partition has the same size
    for _ in range(size):
        index = rm.randint(0, 1) #Random between 0 and 1
        partition_size[index] -=1 #Decrease the maximun amount of nodes in a partition
        if partition_size[index] < 0: #We've already put as much nodes as we can in this partition
            candidate.append(abs(index-1)) #We put this node in the contrary partition
        else:
            candidate.append(index)
    return candidate
            
        
        
        
    

In [5]:
print(generate_binary_balanced_solution(64))

[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0]


A continuación implementa la función objetivo

In [6]:
def objective_function_GPP(candidate, instance):
    size, W = instance
    fitness = 0
   # TODO - 4 lineas aprox
    for i in range(size):
        for j in range(size):
            fitness += candidate[i] * (1 - candidate[j]) * W[i][j] #Función de fitness
                
    return fitness

In [7]:
# Evalua el fitness de una solución
instance = instanceGrid
candidate = generate_binary_balanced_solution(instance[0])
print("Funcion objetivo: {}".format(objective_function_GPP(candidate, instance)))


Funcion objetivo: 45.0


## Variable Neighborhood Descent (VND)


### Estimación de óptimos locales

Vamos a seguir los métodos estudiados en la segunda unidad didáctica para realizar una estimación profesional del número de óptimos locales.
Recuerda que estas técnicas se basan en la idea de generar $m$ candidatos aleatorios y comprobar el porcentaje real de óptimos locales $r$ mediante el ratio $r/m$.

La intuición nos dice que cuantos más óptimos locales diferentes encontremos el problema será más complejo ya que el landscape es más agresivo.

A continuación tu tarea consiste en implementar las siguientes funciones de vecindario:

In [62]:
def hamming (candidate: list):
    vecinos = []
    for i in range(len(candidate)):
        for j in range(len(candidate)):
            vecino = np.copy(candidate)
            if vecino[i] != vecino[j]: #Si los nodos no estan en la misma partición
                vecino[i] = vecino[j]
                vecino[j] = abs(1-vecino[j])
                #print(vecino)
                #print("\n")
                vecinos.append(vecino)
    return vecinos

In [70]:
def local_search_hamming (candidate:list, instance, max_evals : int):
    # TODO aprox 15-20 lineas
    print("        HAMMING    \n ======================")
    mejora = True
    
    best_fitness = objective_function_GPP(candidate, instance)
    best_candidate = candidate
    print("    CURRENT SOLUTION  \n ------------------------------- \n Candidate: {} \n Fitness: {} \n".format(best_candidate, best_fitness))
    
    consumed_evals = 1 #Iniciamos a 1 porque hemos evaluado al candidato inicial
    
    while (mejora and consumed_evals <= max_evals):
        mejora = False
        vecinos = hamming(candidate) #Creamos vecinos mediante hamming
        
        for vecino in vecinos: #Nos paramos cuando encontremos el primero que sea mejor (NOTA: No se si hay que evaluarlos todos o no, de momento lo dejo asi)
            new_fitness = objective_function_GPP(vecino, instance) #Evaluamos el vecino seleccionado
            #print(vecino)
            #print(new_fitness)
            if new_fitness <= best_fitness: #Hemos encontrado una solucion mejor
                best_candidate = vecino #El vecino se convierte en el mejor candidato
                best_fitness = new_fitness #El fitness del vecino se convierte en el mejor fitness
                mejora = True
                #break #Salimos del for (NOTA: No es necesario si hay que evaluar todos los vecino)
            
            #else:
             #   mejora = False #El vecino actual no es mejor que el candidato actual
         
        consumed_evals += 1 #Sumamos 1 al total de soluciones evaluadas
        
        if not mejora:
            print("    NO BETTER SOLUTION FOUND  \n ------------------------------- \n Best candidate: {} \n Best fitness: {} \n Number of evals: {} \n".format(best_candidate, best_fitness, consumed_evals))
            #break #Terminamos el bucle while, pues no hemos encontrado un vecino que sea mejor
        
        else:
            print("    NEW BEST SOLUTION FOUND  \n ------------------------------- \n New best candidate: {} \n New best fitness: {} \n".format(best_candidate, best_fitness))
    

    return (best_fitness, best_candidate, consumed_evals)

In [71]:
# Prueba la funcion local_search_hamming con valores de max_evals diferentes.
# Asegurate de que la implementación es correcta
candidate = generate_binary_balanced_solution(instance[0])
_, _, _ = local_search_hamming (candidate, instance, 10000)

        HAMMING    
    CURRENT SOLUTION  
 ------------------------------- 
 Candidate: [0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0] 
 Fitness: 55.0 

    NEW BEST SOLUTION FOUND  
 ------------------------------- 
 New best candidate: [0 1 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 1
 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 0 0 0] 
 New best fitness: 49.0 

    NEW BEST SOLUTION FOUND  
 ------------------------------- 
 New best candidate: [0 1 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 1
 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 0 0 0] 
 New best fitness: 49.0 

    NEW BEST SOLUTION FOUND  
 ------------------------------- 
 New best candidate: [0 1 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 1
 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 0 0 0

KeyboardInterrupt: 

In [11]:
def insert(candidate: list, instance):
    n = np.size(instance)
    vecinos = []
    for i in range(n):
        for j in range(n):
            vecino = np.copy(candidate)
            vecino = np.insert(vecino, i,candidate[j])
            vecinos.append(vecino)
    return vecinos
    

In [32]:
def local_search_insert (candidate, instance, max_evals):
    # TODO aprox 15-20 lineas
    print("        INSERT    \n ======================")
    mejora = True
    
    best_fitness = objective_function_GPP(candidate, instance)
    best_candidate = candidate
    print("    CURRENT SOLUTION  \n ------------------------------- \n Candidate: {} \n Fitness: {} \n".format(best_candidate, best_fitness))
    
    consumed_evals = 1 #Iniciamos a 1 porque hemos evaluado al candidato inicial
    
    while (mejora and consumed_evals <= max_evals):
        mejora = False
        vecinos = insert(candidate, instance) #Creamos vecinos mediante insert
        #print(vecinos)
        
        for vecino in vecinos: #Nos paramos cuando encontremos el primero que sea mejor (NOTA: No se si hay que evaluarlos todos o no, de momento lo dejo asi)
            new_fitness = objective_function_GPP(vecino, instance) #Evaluamos el vecino seleccionado
            
            if new_fitness < best_fitness: #Hemos encontrado una solucion mejor
                best_candidate = vecino #El vecino se convierte en el mejor candidato
                best_fitness = new_fitness #El fitness del vecino se convierte en el mejor fitness
                mejora = True
                #break #Salimos del for (NOTA: No es necesario si hay que evaluar todos los vecino)
            
            #else:
             #   mejora = False #El vecino actual no es mejor que el candidato actual
         
        consumed_evals += 1 #Sumamos 1 al total de soluciones evaluadas
        
        if not mejora:
            print("    NO BETTER SOLUTION FOUND  \n ------------------------------- \n Best candidate: {} \n Best fitness: {} \n Number of evals: {} \n".format(best_candidate, best_fitness, consumed_evals))
            #break #Terminamos el bucle while, pues no hemos encontrado un vecino que sea mejor
        
        else:
            print("    NEW BEST SOLUTION FOUND  \n ------------------------------- \n New best candidate: {} \n New best fitness: {} \n".format(best_candidate, best_fitness))
    

    return (best_fitness, best_candidate, consumed_evals)

In [21]:
# Prueba la funcion local_search_insert con valores de max_evals diferentes.
# Asegurate de que la implementación es correcta
candidate = generate_binary_balanced_solution(instance[0])
local_search_insert (candidate, instance, max_evals = 10000)

        INSERT    
    CURRENT SOLUTION  
 ------------------------------- 
 Candidate: [1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1] 
 Fitness: 57.0 

    NEW BEST SOLUTION FOUND  
 ------------------------------- 
 New best candidate: [1 0 0 1 0 1 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 1 1 1 0 0 1 1 0 0 0 1 1 0 0 1 0
 1 1 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1] 
 New best fitness: 56.0 

    NO BETTER SOLUTION FOUND  
 ------------------------------- 
 Best candidate: [1 0 0 1 0 1 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 1 1 1 0 0 1 1 0 0 0 1 1 0 0 1 0
 1 1 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1] 
 Best fitness: 56.0 
 Number of evals: 3 



/home/oier/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3202: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).size


(56.0,
 array([1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
        1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 3)

In [65]:
import numpy as np
import time

# Vamos a realizar una pequeña experimentación en un entorno pequeño, como lo es Grid8x8
# para tener una idea del rendimiento de cada aproximación

repetitions = 20
instance = instanceGrid
max_evals = 10000
list_fitness_hamming = []
list_fitness_insert = []

for rep in range(repetitions):
    t0 = time.time()
    np.random.seed(rep)
    candidate = generate_binary_balanced_solution(instance[0])

    ls_hamming_result = local_search_hamming(candidate, instance, max_evals)
    if ls_hamming_result[2] < max_evals:
        list_fitness_hamming.append(ls_hamming_result[0])

    ls_insert_result = local_search_insert(candidate, instance, max_evals)
    if ls_insert_result[2] < max_evals:
        list_fitness_insert.append(ls_insert_result[0])

    print("Repetition #: {}. Elapsed time: {} \n".format(rep, time.time() - t0))

print("")
print("Hamming Total Local Optima #: {}".format(len(list_fitness_hamming)))
print("Ratio of total Local optima under Hamming: {}".format(len(list_fitness_hamming) / (rep + 1)))
print("Hamming Different Local Optima #: {}".format(len(np.unique(list_fitness_hamming))))
print("Ratio of Different Local optima under Hamming: {}".format(len(np.unique(list_fitness_hamming)) / (rep + 1)))
print("")
print("Insert Total Local Optima #: {}".format(len(list_fitness_insert)))
print("Ratio of total local optima under Insert: {}".format(len(list_fitness_insert) / (rep + 1)))
print("Insert Different Local Optima #: {}".format(len(np.unique(list_fitness_insert))))
print("Ratio of Different local optima under Insert: {}".format(len(np.unique(list_fitness_insert)) / (rep + 1)))

        HAMMING    
    CURRENT SOLUTION  
 ------------------------------- 
 Candidate: [0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1] 
 Fitness: 64.0 

    NEW BEST SOLUTION FOUND  
 ------------------------------- 
 New best candidate: [0 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 1 0 1 1 1 1 0 1 0
 1 0 0 0 0 0 1 0 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 1 1 1 1] 
 New best fitness: 56.0 

    NO BETTER SOLUTION FOUND  
 ------------------------------- 
 Best candidate: [0 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 1 0 1 1 1 1 0 1 0
 1 0 0 0 0 0 1 0 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 1 1 1 1] 
 Best fitness: 56.0 
 Number of evals: 3 

        INSERT    
    CURRENT SOLUTION  
 ------------------------------- 
 Candidate: [0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 

/home/oier/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3202: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).size


    NEW BEST SOLUTION FOUND  
 ------------------------------- 
 New best candidate: [0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 1 1 0 1 1 0 0
 0 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1] 
 New best fitness: 47.0 

    NO BETTER SOLUTION FOUND  
 ------------------------------- 
 Best candidate: [0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 1 1 0 1 1 0 0
 0 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1] 
 Best fitness: 47.0 
 Number of evals: 3 

        INSERT    
    CURRENT SOLUTION  
 ------------------------------- 
 Candidate: [0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1] 
 Fitness: 53.0 

    NEW BEST SOLUTION FOUND  
 ------------------------------- 
 New best candidate: [0 1 1 1 1 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 0 1 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0
 0 0 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1 

¿ Por qué es relevante confirmar el número de iteraciones realizadas para registrar un óptimo local ?

Vistos y analizados estos porcentajes, ¿Cuál crees que debería ser la función de vecindario principal? y ¿cuál la auxiliar?

### VND vs. Basic local search

Una vez que hemos realizado los ajustes necesarios, implementa el algoritmo VND siguiente la siguiente declaración

In [ ]:
def VND_for_GPP(instance : list, max_evals = 10000):
    # TODO 15-20 lineas

    return (best_fitness, best_candidate, consumed_evals)

In [ ]:
import time as tm

# Cargar instancia
instance = read_instance_GPP("G124.16")

# Ejecutar algoritmo y medir los tiempos de ejecución
start = tm.time()
max_evals = 100000
fitness, sol, consumed_evals = VND_for_GPP(instance, max_evals)
end = tm.time()

# Imprimir valores
print("Best fitness: {}, solution: {}".format(fitness, sol))
print("Execution time: {}".format(end-start))
print("Evaluations consumed: {}".format(consumed_evals))

In [28]:
# Experimento para evaluar nuestro algoritmo VND acorde con el número total de iteraciones

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

list_iterations = []
list_avg_fitness = []
repetitions = 10

# Cargamos instancia
instance = instanceG124

# realizar experimentación con distintos hiperparametros: 10, 100, 1000, 10000...

for exp in range(1,6):

    iteraciones = 10**exp
    avg_fit = 0

    for rep in range(repetitions):

        np.random.seed(rep)
        fitness, sol, consumed_iterations = VND_for_GPP(instance, rep)
        avg_fit += fitness

    avg_fit /= repetitions
    list_iterations.append(iteraciones)
    list_avg_fitness.append(avg_fit)


NameError: name 'VND_for_GPP' is not defined

In [ ]:
datos = pd.DataFrame( {"iteraciones": list_iterations,
                       "AVG fitness": list_avg_fitness })

In [ ]:
print(datos.head())
print(datos.size)
print(datos)

In [ ]:
datos.set_index('iteraciones', inplace = True)
datos.head()

In [ ]:
%matplotlib inline
datos.plot(kind = 'line', ylim=(0,1000), use_index = True)

¿ Qué conclusiones puedes extraer después de analizar toda esta experimentación realizada ?
¿ Te atreves a comparar y graficar el rendimiento de VND vs random search ?
¿ Cuáles serían los pasos para transformar VND en VNS ?

In [ ]:
# Indica aquí tus conclusiones y lecciones aprendidas